<a href="https://colab.research.google.com/github/Barleysack/tensor101/blob/handsonml2_9/Attention%26RNN_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)




In [48]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [49]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [50]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text) #여기서 인덱스가 1~39로 나타난다.

In [51]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [52]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

#이 토크나이저 api는 이후 프로젝트에서 출력시 사용 가능해보인다. 
#가능할까? 


['f i r s t']

In [53]:
max_id = len(tokenizer.word_index) # 고유 글자 개수.
dataset_size = tokenizer.document_count # 총 글자 개수.


In [54]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1 #여기서 0~38까지로 알아듣기 위해 1을 뺌.#encoded 배열은...이라는 뜻. 배열을 지정하는 법을 여기서 배우다니 조금 늦은 감이 없잖아 있지 않나 싶다.
train_size = dataset_size * 90 // 100 #처음 90퍼센트를 훈련 세트로 사용한다.
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size]) #시계열 데이터는 변하지 않으리라 가정한다. 그렇지 않은 경우도 많다. encoded 배열의 원소를 글자별로 나눠 텐서로 반환하는 tf.data.Dataset 객체 생성

In [55]:
n_steps = 100
window_length = n_steps + 1 # 타겟은 한 글자 앞의 INPUT, #window 메소드는 각각 하나의 데이터셋으로 표현되는 윈도를 포함하는 데이터셋을 만들어냄. 리스트의 리스트 개념이라고 보면 된다. 중첩 데이터셋의 생성.
"""[1,2,3,4,5,6,7,8,9,10]이 있을때 윈도우 크기가 3이라면, 하기된 메서드는 [1,2,3],[2,3,4],[3,4,5].....하는 식으로 된 데이터셋을 생성하는 메서드이다."""

dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)#다른 데이터와 크기가 다른 윈도는 모두 쳐냄.
#하지만 모델은 텐서를 받아야한다. 데이터셋이 아니라. 따라서 플랫-데이터셋으로 변환한다. 

In [56]:
dataset = dataset.flat_map(lambda window: window.batch(window_length)) #플랫 데이터셋으로 변환. 윈도마다 batch(window_length)를 호출하여 윈도 길이와 같은 텐서 하나를 다믕ㄴ 데이터셋을 얻는다. 따라서 101글자...!
#경사하강법은 훈련세트 샘플이 동일 독립 분포일때 가장 잘 작동한다. 셔플 후 배치로 만들고 난 뒤

In [57]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size) #데이터셋을 셔플한 뒤 배치로 변환한다. 
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))#입력(윈도우의 처음 100글자)와 타깃(마지막 글자)를 분리해준다. 

In [58]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))#범주형 입력 특성이기에 원-핫 벡터로 인코딩한다.
    

In [60]:
dataset = dataset.prefetch(1) #프리페칭 추가. 
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


# 이로서 데이터셋 준비 완료. 모델을 만들고 훈련한다. 
#사용 모델은 Char-RNN


In [1]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2), #recurrent_dropout=0.2), #recurrent dropout? 입력과 은닉상태 모두에게 20퍼센트 드롭아웃을 사용하겠다는 뜻. RNN의 구조에 대한 복습 필요. 128개 유닛을 가진 GRU 2개층 사용. 
                  
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2), #recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)
                                 

NameError: ignored

In [ ]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["How are yo"])
#Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model.predict(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

In [ ]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

In [ ]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1)